In [33]:
import arcpy
from arcpy import *
#https://app.diagrams.net/

output_folder = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb"


arcpy.env.workspace = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb"
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
arcpy.env.overwriteOutput = True
arcpy.env.extent = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\swieradow_zdroj_buffer")
arcpy.env.mask = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\swieradow_zdroj_buffer")
arcpy.env.cellSize = (R"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\input_data\gmina\raster_extracted.tif")

aprx = arcpy.mp.ArcGISProject("CURRENT") 
m = aprx.activeMap

# wylaczenie dodawania warstw
arcpy.env.addOutputsToMap = False



In [77]:
drogi_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\SKDR.shp")
rzeki = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\SWRS.shp")
jeziora = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\PTWP.shp")
lasy = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\PTLZ.shp")
budynki_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\BUBD")
bufor = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\swieradow_zdroj_buffer")
raster_gmina = (R"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\input_data\gmina\raster_extracted.tif")
gmina = (R"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\input_data\gmina\swieradow_zdroj.shp")
linie_energ = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\analiza_polozenia_farmy\analiza_polozenia_farmy\analiza_polozenia_farmy.gdb\SULN.shp")
# warstwa z pokryciem terenu
pokrycie = R"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\input_data\pokrycie_terenu\mapa_kosztow_BDOT10k_dane_archiwalne\PT_merge_cliped.shp"

# działki 
dzialki = R"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\dzialki.shp"
arcpy.management.CheckGeometry(budynki_wszystkie, "in_memory/geometry_issues")


<Result 'in_memory\\geometry_issues'>

In [39]:
#wyselekcjonowanie drog utwardzonych
drogi = arcpy.management.SelectLayerByAttribute(drogi_wszystkie, "NEW_SELECTION", "MATE_NAWIE IN ('beton','bruk', 'kostka kamienna', 'kostka prefabrykowana', 'masa bitumiczna', 'płyty betonowe')", None)
arcpy.management.CopyFeatures(drogi, f"{output_folder}\drogi_selected", '', None, None, None)
drogi_selected = 'drogi_selected'
arcpy.SelectLayerByAttribute_management(drogi, "CLEAR_SELECTION")

#wyselekcjonowanie budynkow mieszkalnych
budynki = arcpy.management.SelectLayerByAttribute(budynki_wszystkie, "NEW_SELECTION", "FOBUD IN ('budynki mieszkalne')", None)
arcpy.management.CopyFeatures(budynki, f"{output_folder}\budynki_selected", '', None, None, None)
budynki_selected = 'budynki_selected'
arcpy.SelectLayerByAttribute_management(budynki, "CLEAR_SELECTION")
print("ok")

ok


In [40]:
distance_rzeki = arcpy.sa.EucDistance(rzeki, None, "5")
distance_jeziora = arcpy.sa.EucDistance(jeziora, None, "5")
distance_drogi = arcpy.sa.EucDistance(drogi_selected, None, "5")
distance_budynki = arcpy.sa.EucDistance(budynki_selected, None, "5")
distance_lasy = arcpy.sa.EucDistance(lasy, None, "5")

# mapa odleglosci do kryterium z wezlami wykonana w QGiS:
distance_wezly  = Raster(r"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\kryterium_wezly_komunikacyjne.tif")

## Realizacja kryteriów

Kryterium 1 - odległość od rzek i zbiorników wodnych, jak najbliżej (nieprzekraczalna strefa ochronna 100m)

In [41]:
#kryterium ostre dla strefy ochronnej 100m poprzez reklasyfikacje map odleglosci
rzeki_ostre = arcpy.ddd.Reclassify(distance_rzeki, "VALUE", f"0 100 0;100 {distance_rzeki.maximum} 1")
jeziora_ostre = arcpy.ddd.Reclassify(distance_jeziora, "VALUE", f"0 100 0;100 {distance_jeziora.maximum} 1")

#połączenie kryteriów ostrych dla rzek i jezior
wody_ostre = arcpy.sa.FuzzyOverlay([rzeki_ostre, jeziora_ostre], "AND")

In [42]:
#kryterium rozmyte dla obszarów w odległości powyżej 100m
rzeki_rozmyte = arcpy.sa.FuzzyMembership(distance_rzeki, arcpy.sa.FuzzyLinear(500, 200))
jeziora_rozmyte = arcpy.sa.FuzzyMembership(distance_jeziora, arcpy.sa.FuzzyLinear(500, 200))

#połączenie kryteriów rozmytych dla rzek i jezior
wody_rozmyte = arcpy.sa.FuzzyOverlay([rzeki_rozmyte, jeziora_rozmyte], "OR")

In [43]:
#połączenie kryterium ostrego i rozmytego dla wody
kryterium_wody = arcpy.sa.FuzzyOverlay([wody_ostre, wody_rozmyte], 'AND')
kryterium_wody.save(f"{output_folder}\kryterium_wody")

Kryterium 2 - dostęp do dróg utwardzonych, jak największe zagęszczenie 

In [44]:
# ocena przydatnosci pod wzgledem zageszczenia
raster_zageszczenia = arcpy.sa.LineDensity(
    in_polyline_features=drogi_selected,
    population_field="NONE",
    cell_size=5,
    search_radius=1000,
    area_unit_scale_factor="SQUARE_METERS"
)
arcpy.management.CalculateStatistics(raster_zageszczenia)

max_value = raster_zageszczenia.maximum

# przeskalowanie rastra zageszczenia do wartosci 0-1
kryterium_drogi = arcpy.sa.RescaleByFunction(
    in_raster=raster_zageszczenia,
    transformation_function= f"LINEAR 0 1,{max_value} 0 # {max_value} #",
    from_scale=0,
    to_scale=1
)

kryterium_drogi.save(f"{output_folder}\kryterium_drogi")


Kryterium 3 - odległość od budynków mieszkalnych

In [46]:
kryterium_budynki = arcpy.sa.FuzzyMembership(distance_budynki,  arcpy.sa.FuzzyLinear(150, 500))
kryterium_budynki.save(f"{output_folder}\kryterium_budynki")

Kryterium 4 - pokrycie terenu, nie w lesie, powyżej 15m od lasu, optymalnie 100m od lasu 

In [47]:
kryterium_lasy = arcpy.sa.FuzzyMembership(distance_lasy, arcpy.sa.FuzzyLinear(15, 100))
kryterium_lasy.save(f"{output_folder}\kryterium_lasy")

Kryterium 5 - nachylenie stoków, optymalnie płasko, maksymalnie 10%

In [50]:
#stworzenie warstwy z nachyleniami stoków narzędziem Slope
spadki_output = f"{output_folder}\spadki"
spadki = arcpy.ddd.Slope(raster_gmina, spadki_output, "PERCENT_RISE", 1)

kryterium_spadki = arcpy.sa.FuzzyMembership(spadki, arcpy.sa.FuzzyLinear(10, 3))
kryterium_spadki.save(f"{output_folder}\kryterium_spadki")

Kryterium 6 - nasłonecznienie, optymalnie stoki południowe (SW-SE)

In [51]:
#stworzenie warstwy ze stokami narzędziem Aspect
stoki_output = f"{output_folder}\stoki"
stoki = arcpy.ddd.Aspect(raster_gmina, stoki_output)

# rozdzielenie kryterium rozmytego na rosnący i malejący
stoki_ES = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(90, 113))
stoki_WS = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(270, 248))

# ustawienie przydatności płaskich stoków na 1
plaskie_stoki = arcpy.sa.Con(stoki, 1, 0, "VALUE = -1") 

stoki_rozmyte = arcpy.sa.FuzzyOverlay([stoki_ES, stoki_WS], 'AND')
kryterium_stoki = arcpy.sa.FuzzyOverlay([stoki_rozmyte, plaskie_stoki], 'OR')

kryterium_stoki.save(f"{output_folder}\kryterium_stoki")

Kryterium 7 - dobry dojazd od istotnych drogowych węzłów komunikacyjnych

In [52]:
kryterium_wezly = arcpy.sa.FuzzyMembership(distance_wezly, arcpy.sa.FuzzyLinear(distance_wezly.maximum, 300))
kryterium_wezly.save(f"{output_folder}\kryterium_wezly")
m.addDataFromPath(f"{output_folder}\kryterium_wezly")

## Połączenie kryteriów

In [53]:
# połączenie kryteriów stosując równe wagi dla kryteriów
rowna_waga = 1/7
kryteria_rowne = arcpy.sa.WSTable(
    [
        [kryterium_budynki, "VALUE", rowna_waga],
        [kryterium_drogi, "VALUE", rowna_waga],
        [kryterium_lasy, "VALUE", rowna_waga],
        [kryterium_spadki, "VALUE", rowna_waga],
        [kryterium_stoki, "VALUE", rowna_waga],
        [kryterium_wody, "VALUE", rowna_waga],
        [kryterium_wezly, "VALUE", rowna_waga],
    ]
)
polaczone_kryteria_rowne = arcpy.sa.WeightedSum(kryteria_rowne)
polaczone_kryteria_rowne.save(f"{output_folder}\polaczone_kryteria_rowne")

# połączenie kryteriów stosując różne wagi dla kryteriów metodą AHP : https://bpmsg.com/ahp/ahp-calc.php?n=7&t=AHP+priorities&c[0]=odleglosc+od+wod&c[1]=odleglosc+od+lasow&c[2]=odl+bud&c[3]=drogi&c[4]=spadki&c[5]=naslonecznienie&c[6]=wezly
kryteria_rozne = arcpy.sa.WSTable(
    [
        [kryterium_budynki, "VALUE", 0.120244],
        [kryterium_drogi, "VALUE", 0.051404],
        [kryterium_lasy, "VALUE", 0.248666],
        [kryterium_spadki, "VALUE", 0.187234],
        [kryterium_stoki, "VALUE", 0.163683],
        [kryterium_wody, "VALUE", 0.207279],
        [kryterium_wezly, "VALUE", 0.021489],
    ]
)
polaczone_kryteria_rozne = arcpy.sa.WeightedSum(kryteria_rozne)
polaczone_kryteria_rozne.save(f"{output_folder}\polaczone_kryteria_rozne")
print("ok")


ok


## Uwzględnienie kryteriów ostrych do map przydatności (nieprzekraczalne strefy w kryterium wody, budynków i lasów)

In [55]:
# budynki ostre:
budynki_ostre = arcpy.ddd.Reclassify(distance_budynki, "VALUE", f"0 150 0; 150 {distance_budynki.maximum} 1")

# lasy ostre:
lasy_ostre = arcpy.ddd.Reclassify(distance_lasy, "VALUE", f"0 15 0; 15 {distance_lasy.maximum} 1")

def times(in_raster1, in_raster2):
    out_raster = arcpy.ia.Times(
        in_raster_or_constant1=in_raster1,
        in_raster_or_constant2=in_raster2
    )
    return out_raster

# przemnożenie mapy przydatności przez kryteria ostre:
def polacz_ostre(polaczone_kryteria):
    mapa_przydatnosci1 = times(polaczone_kryteria, wody_ostre)
    mapa_przydatnosci2 = times(mapa_przydatnosci1, budynki_ostre)
    mapa_przydatnosci = times(mapa_przydatnosci2, lasy_ostre)

    return mapa_przydatnosci

mapa_przydatnosci_rowne = polacz_ostre(polaczone_kryteria_rowne)
mapa_przydatnosci_rozne = polacz_ostre(polaczone_kryteria_rozne)

mapa_przydatnosci_rowne.save(f"{output_folder}\mapa_przydatnosci_rowne")
# m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_rowne")
mapa_przydatnosci_rozne.save(f"{output_folder}\mapa_przydatnosci_rozne")
# m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_rozne")

## Reklasyfikacja map przydatnosci w celu wyodrębnienia obszarów o 80% przydatności

In [56]:
def reklasyfikuj_mape(mapa_przydatnosci):
    granica = 70/100 * mapa_przydatnosci.maximum
    mapa_reclass = arcpy.sa.Reclassify(mapa_przydatnosci, "VALUE", f"0 {granica} 0; {granica} {mapa_przydatnosci.maximum} 1")
    return mapa_reclass

# wyciecie jedynie do obszaru gminy (bez bufora)
mapa_przydatnosci_rowne = arcpy.sa.ExtractByMask(
    in_raster=mapa_przydatnosci_rowne,
    in_mask_data=gmina,
    extraction_area="INSIDE"
)

mapa_przydatnosci_rozne = arcpy.sa.ExtractByMask(
    in_raster=mapa_przydatnosci_rozne,
    in_mask_data=gmina,
    extraction_area="INSIDE"
)

mapa_przydatnosci_reclass_rowne = reklasyfikuj_mape(mapa_przydatnosci_rowne)
mapa_przydatnosci_reclass_rozne = reklasyfikuj_mape(mapa_przydatnosci_rozne)
mapa_przydatnosci_reclass_rowne.save(f"{output_folder}\mapa_przydatnosci_reclass_rowne")
# m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_reclass_rowne")
mapa_przydatnosci_reclass_rozne.save(f"{output_folder}\mapa_przydatnosci_reclass_rozne")
# m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_reclass_rozne")

## Obliczenie jaka część działki stanowi obszar przydatny pod lokalizację inwestycji

In [57]:
# przekształcenie map przydatności do postaci wektorowej
out_polygon_features = r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_wektor"

def oblicz_przydatnosc_dzialki(mapa_przydatnosci, out_str):
    mapa_przydatnosci_wektor = f"{output_folder}\mapa_przydatnosci_wektor"
    with arcpy.EnvManager(outputZFlag="Disabled", outputMFlag="Disabled"):
        arcpy.conversion.RasterToPolygon(
            in_raster=mapa_przydatnosci,
            out_polygon_features=mapa_przydatnosci_wektor,
            simplify="SIMPLIFY",
            raster_field="Value",
            create_multipart_features="SINGLE_OUTER_PART",
            max_vertices_per_feature=None
        )

    # Intersect aby przyciąć poligony przydatności do działek:
    przydatnosc_dzialki_przeciecie = arcpy.analysis.Intersect(
        in_features=[mapa_przydatnosci_wektor, dzialki],
        out_feature_class=f"{output_folder}\przydatnosc_dzialki_przeciecie",
        join_attributes="ALL",
        cluster_tolerance=None,
        output_type="INPUT"
    )

    # Selekcja tylko przydatnych poligonów 
    przydatne_poligony = arcpy.management.MakeFeatureLayer(
        in_features=przydatnosc_dzialki_przeciecie,
        out_layer=f"{output_folder}\przydatne_poligony",
        where_clause="gridcode = 1",
        workspace=None,
    )

    # Zliczenie jaka część działki jest pokryta przydatnymi obszarami
    przydatnosc_dzialki_pole = arcpy.analysis.SummarizeWithin(
        in_polygons=dzialki,  # Warstwa działek
        in_sum_features=przydatne_poligony,  # Poligony przydatne (gridcode = 1)
        out_feature_class=f"{output_folder}\przydatnosc_dzialki_pole_{out_str}",
        keep_all_polygons="ONLY_INTERSECTING",  
        sum_fields="Shape_Area Sum", 
        shape_unit="SQUAREMETERS"
    )
    
    # dodanie atrybutu o % przydatności działki
    arcpy.management.CalculateField(
        in_table=przydatnosc_dzialki_pole,
        field="Procent_powierzchni_przydatnej",
        field_type = "DOUBLE",
        expression="(!SUM_Shape_Area! / !Shape_Area!) * 100",
        expression_type="PYTHON3"
    )
    
    return przydatnosc_dzialki_pole

przydatnosc_dzialki_rowne = oblicz_przydatnosc_dzialki(mapa_przydatnosci_reclass_rowne, "rowne")
m.addDataFromPath(f"{output_folder}\przydatnosc_dzialki_pole_rowne")

przydatnosc_dzialki_rozne = oblicz_przydatnosc_dzialki(mapa_przydatnosci_reclass_rozne, "rozne")
m.addDataFromPath(f"{output_folder}\przydatnosc_dzialki_pole_rozne")

## Wybranie działek spełniających kryteria - min 70% przydatności

In [58]:
# Selekcja działek o min. 70% przydatności:
def selekcja_przydatnych_dzialek(przydatnosc_dzialki, out_str):
    przydatne_dzialki = arcpy.management.SelectLayerByAttribute(przydatnosc_dzialki, "NEW_SELECTION", '"Procent_powierzchni_przydatnej" > 60', None)
    arcpy.management.CopyFeatures(przydatne_dzialki, f"{output_folder}\przydatne_dzialki_{out_str}", '', None, None, None)
    przydatne_dzialki_selected = 'przydatne_dzialki_selected'
    
    # Połączenie sąsiadujących działek w jeden obiekt
    polaczone_dzialki = arcpy.management.Dissolve(
        in_features=przydatne_dzialki,
        out_feature_class=f"{output_folder}\polaczone_dzialki_{out_str}",
        dissolve_field=None,
        statistics_fields="Shape_Area SUM",
        multi_part="SINGLE_PART",
        unsplit_lines="UNSPLIT_LINES",
        concatenation_separator=""
    )
    
    arcpy.SelectLayerByAttribute_management(przydatne_dzialki, "CLEAR_SELECTION")
    return polaczone_dzialki

polaczone_dzialki_rowne = selekcja_przydatnych_dzialek(przydatnosc_dzialki_rowne, "rowne")
polaczone_dzialki_rozne = selekcja_przydatnych_dzialek(przydatnosc_dzialki_rozne, "rozne")

## Wybranie działek spełniających kryterium powierzchni i szerokości (min 2ha i 50m)

In [60]:
def dodaj_szerokosc(polaczone_dzialki, sciezka_wyjscia):
    # utworzenie prostokątów ograniczających wokół działek 
    prostokaty_ograniczajace = arcpy.management.MinimumBoundingGeometry(
        polaczone_dzialki, 
        sciezka_wyjscia,
        "RECTANGLE_BY_WIDTH")

    arcpy.management.AddField(
        prostokaty_ograniczajace,
        field_name="Width",
        field_type="DOUBLE",
    )

    arcpy.topographic.CalculateMetrics(
        in_features=prostokaty_ograniczajace,
        in_metric_types="WIDTH",
        in_length_attributes="Shape_Length",
        in_width_attributes="Width",
        in_area_attributes="Shape_Area",
    )

    # Wykonanie Join, aby przekazać atrybut szerokości do działek
    fields = arcpy.ListFields(polaczone_dzialki)
    for field in fields:
        if field.name == "Width":
            arcpy.management.DeleteField(polaczone_dzialki, "Width")

    arcpy.management.JoinField(
        in_data=polaczone_dzialki,         
        in_field="OBJECTID",                      
        join_table=prostokaty_ograniczajace,      
        join_field="OBJECTID",                   
        fields=["Width"],                  
    )
    return polaczone_dzialki

polaczone_dzialki_rowne = f"{output_folder}\polaczone_dzialki_rowne"
sciezka_wyjscia_rowne = f"{output_folder}\polaczone_dzialki_rowne_prostokaty"

polaczone_dzialki_rozne = f"{output_folder}\polaczone_dzialki_rozne"
sciezka_wyjscia_rozne = f"{output_folder}\polaczone_dzialki_rozne_prostokaty"

dodaj_szerokosc(polaczone_dzialki_rowne, sciezka_wyjscia_rowne)
dodaj_szerokosc(polaczone_dzialki_rozne, sciezka_wyjscia_rozne)
m.addDataFromPath(f"{output_folder}\polaczone_dzialki_rowne")
m.addDataFromPath(f"{output_folder}\polaczone_dzialki_rozne")


In [61]:
# selekcja działek:
dzialki_rowne_copy = arcpy.management.MakeFeatureLayer(
    in_features=polaczone_dzialki_rowne,
    out_layer=f"{output_folder}\dzialki_rowne_copy",
    where_clause='"Shape_Area" >= 20000 AND "Width" >= 50',
)

dzialki_rowne = arcpy.management.CopyFeatures(
    in_features=dzialki_rowne_copy,  
    out_feature_class=f"{output_folder}\dzialki_rowne"
)

dzialki_rozne_copy = arcpy.management.MakeFeatureLayer(
    in_features=polaczone_dzialki_rozne,
    out_layer=f"{output_folder}\dzialki_rozne_copy",
    where_clause='"Shape_Area" >= 20000 AND "Width" >= 50',
)

dzialki_rozne = arcpy.management.CopyFeatures(
    in_features=dzialki_rozne_copy,  
    out_feature_class=f"{output_folder}\dzialki_rozne"
)

m.addDataFromPath(f"{output_folder}\dzialki_rowne")
m.addDataFromPath(f"{output_folder}\dzialki_rozne")

## Utworzenie mapy kosztów

In [76]:
# Konwersja warstwy wektorowej do rastra po wartości atrybutu X_KOD
raster_pokrycia = arcpy.conversion.FeatureToRaster(
    in_features= pokrycie,
    field="X_KOD",
    out_raster="raster_pokrycia",
    cell_size=5
)

# Reklasyfikacja w celu przypisania kosztów dla różnych wartości X_KOD
raster_kosztow = arcpy.ddd.Reclassify(
    in_raster=raster_pokrycia,
    reclass_field="X_KOD",
    remap="PTZB01 200;PTZB02 100;PTZB05 50;PTZB03 200;PTZB04 200;PTWZ01 NODATA;PTWP02 200;PTWP03 NODATA;PTUT03 100;PTUT01 NODATA;PTTR02 1;PTTR01 20;PTPL01 50;PTNZ01 150;PTNZ02 150;PTLZ01 100;PTLZ03 50;PTLZ02 50;PTKM01 100;PTWP01 NODATA;PTRK01 15;PTRK02 15;PTUT02 90;PTUT04 20;PTUT05 20;PTKM02 200;PTKM03 200;PTKM04 NODATA;PTGN01 1;PTGN02 1;PTGN03 1;PTGN04 1;PTSO01 NODATA;PTSO02 NODATA;PTWZ02 NODATA",
    out_raster="raster_kosztow",
    missing_values="NODATA"
)

In [80]:

with arcpy.da.SearchCursor(dzialki_rowne,['SHAPE@', 'OBJECTID']) as cursor:
    for row in cursor:
        geometry = row[0]
        object_id = row[1]
        
        warstwa = f"in_memory\\dzialka{object_id}_rowne"
        arcpy.management.CopyFeatures([geometry], warstwa)
            
        # Zapisanie warstwy jako pliku w folderze wyjściowym
        sciezka_wyjsciowa = (f"{output_folder}\dzialka_{object_id}_rowne")
        dzialka = arcpy.management.CopyFeatures(warstwa, sciezka_wyjsciowa)
        
        mapa_kosztow = arcpy.sa.CostDistance(
            in_source_data=dzialka,
            in_cost_raster=raster_kosztow,
            maximum_distance=None,
            out_backlink_raster=f"{output_folder}\mapa_kierunkow_{object_id}_rowne"
        )
        mapa_kosztow.save(f"{output_folder}\mapa_kosztow_{object_id}_rowne")

        mapa_kierunkow = f"{output_folder}\mapa_kierunkow_{object_id}_rowne"

#         m.addDataFromPath(f"{output_folder}\mapa_kierunkow_wszystkie_rowne")
#         m.addDataFromPath(f"{output_folder}\mapa_kosztow_wszystkie_rowne")

        out_raster = arcpy.sa.CostPath(
            in_destination_data=linie_energ,
            in_cost_distance_raster=mapa_kosztow,
            in_cost_backlink_raster=mapa_kierunkow,
            path_type="BEST_SINGLE",
            destination_field="OBJECTID",
            force_flow_direction_convention="INPUT_RANGE"
        )

        # Zapisanie wyniku
        out_raster.save(f"{output_folder}\cost_path_{object_id}_rowne")
        m.addDataFromPath(f"{output_folder}\cost_path_{object_id}_rowne")